In [ ]:
import numpy as np

from meshcat import Visualizer
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial, TriangularMeshGeometry, LineSegments, LineBasicMaterial, PointsGeometry
from meshcat.transformations import translation_matrix, rotation_matrix
from matplotlib.colors import to_hex
from pydrake.all import HPolyhedron, Hyperrectangle
import mcubes
from  pydrake.all import ( 
                          Rgba,TriangleSurfaceMesh, SurfaceTriangle,
                          MathematicalProgram, SnoptSolver, le, Iris)
import colorsys
import itertools
from fractions import Fraction
import random
from scipy.spatial import ConvexHull
from pydrake.all import VPolytope
from pydrake.planning import IrisFromCliqueCover, IrisFromCliqueCoverOptions, IrisInConfigurationSpaceFromCliqueCover

def infinite_hues():
    yield Fraction(0)
    for k in itertools.count():
        i = 2**k # zenos_dichotomy
        for j in range(1,i,2):
            yield Fraction(j,i)

def hue_to_hsvs(h: Fraction):
    # tweak values to adjust scheme
    for s in [Fraction(6,10)]:
        for v in [Fraction(6,10), Fraction(9,10)]:
            yield (h, s, v)

def rgb_to_css(rgb) -> str:
    uint8tuple = map(lambda y: int(y*255), rgb)
    return tuple(uint8tuple)

def n_colors(n=33, rgbs_ret = False):
    hues = infinite_hues()
    hsvs = itertools.chain.from_iterable(hue_to_hsvs(hue) for hue in hues)
    rgbs = (colorsys.hsv_to_rgb(*hsv) for hsv in hsvs)
    csss = (rgb_to_css(rgb) for rgb in rgbs)
    to_ret = list(itertools.islice(csss, n)) if rgbs_ret else list(itertools.islice(csss, n))
    return to_ret

def n_colors_random(n=33, rgbs_ret = False):
    colors = n_colors(100 * n, rgbs_ret)
    return random.sample(colors, n)

class GridWorld(Visualizer): # Assuming 2D? Because village is 3D?

    def __init__(self):
        super().__init__()
        self['/Background'].set_property('visible', False)
        # self['/Grid'].set_property('visible', False)

    def build(self, x_bound=10, y_bound=10, density=0.3, seed=7):
        np.random.seed(seed)
        self.x_bound = x_bound
        self.y_bound = y_bound
        self.density = density
        self.boundary = Hyperrectangle((0, 0), (self.x_bound, self.y_bound)) # Polyhedron
        self.area = self.x_bound * self.y_bound

        rng = np.random.default_rng()
        self.obstacle_inds = rng.choice(self.area, size=int(self.area * density), replace=False)
        self.obstacles = []
        self.grid= np.zeros((self.area))
        self.grid[self.obstacle_inds] = 1

        for i, obstacle in enumerate(self.obstacle_inds):
            index = self.ind_to_coords(obstacle)
            center = np.append(np.copy(index), 0) + 0.5
            _box(self, f"obstacles/{i}", center, 1, 1, (0, 0, 0), 1)
            self.obstacles.append(Hyperrectangle(index, index+1))
        # _box(self, f"boundary", (x_bound/2, y_bound/2, 0.5), x_bound, y_bound, (0, 0, 0), 0.95, d=0, wireframe=True)

    def ind_to_coords(self, index):
        return np.array((index % self.x_bound, index // self.y_bound))

    def plot_regions(self, regions, ellipses = None,
                        region_suffix = '', colors = None,
                        wireframe = False,
                        opacity = 0.7,
                        fill = True,
                        line_width = 10,
                        darken_factor = .2,
                        el_opacity = 0.3):
        if colors is None:
            colors = n_colors_random(len(regions), rgbs_ret=True)

        for i, region in enumerate(regions):
            c = tuple([col/255 for col in colors[i]])#,opacity)
            prefix = f"/cfree/regions{region_suffix}/{i}"
            name = prefix# + "/hpoly"
            if region.ambient_dimension() == 3:
                self.plot_hpoly3d_2(name, region,
                                    c, wireframe = wireframe,opacity = opacity)

def _box(vis, name, c, w, h, color, opacity, d=1, wireframe=False):
    c = np.array(c, dtype=float)
    color = to_hex(color)
    material = MeshLambertMaterial(color=color, opacity=opacity, wireframe=wireframe)
    cube = vis[name]
    cube.set_object(Box((w, h, d)), material)
    cube.set_transform(translation_matrix(c))
    return cube

world = GridWorld()
x_bound = 2 #change village side here
y_bound = 2
world.build(x_bound=x_bound, y_bound=y_bound, density=0.5) #, village_side=village_side, building_every=5, density=0.15, seed=seed)

In [ ]:
# Requires HPolyhedrons, not Hyperrectangles
options = IrisFromCliqueCoverOptions()
options.num_builders = 1
options.num_points_per_coverage_check = 2
options.num_points_per_visibility_round = 2

sets = IrisFromCliqueCover(obstacles=world.obstacles, domain=world.boundary.MakeHPolyhedron(), options=options, sets=[])
print(sets)